In [1]:
import importlib

import scripts.data_prep
import scripts.trainer
import scripts.train
import scripts.verification
import models.ResNeXt
import scripts.visual_pics
importlib.reload(scripts.visual_pics) 
importlib.reload(models.ResNeXt) 
importlib.reload(scripts.verification) 
importlib.reload(scripts.data_prep) 
importlib.reload(scripts.trainer) 
importlib.reload(scripts.train) 

from scripts.verification import valid_epoch_ver
from scripts.test import test_epoch_ver
from scripts.data_prep import create_transforms, ImageDataset, ImagePairDataset
from scripts.trainer import Trainer
from scripts.visual_pics import show_cls_dataset_samples, show_ver_dataset_samples
from scripts.train import train_epoch
from models.ResNeXt import ResNeXt, initialize_weights
from customs.arcface_loss import Arcface
from models.temp import Network

import torch
import torch.nn as nn
from torchsummary import summary
import torchvision
from torchvision.utils import make_grid
from torchvision import transforms
import torchvision.transforms.v2 as T
from torch.utils.data import DataLoader
import torch.nn.functional as F
import os
import gc
from tqdm import tqdm
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn import metrics as mt
from scipy.optimize import brentq
from scipy.interpolate import interp1d
import glob
import wandb
import matplotlib.pyplot as plt
from pytorch_metric_learning import samplers
import csv

DEVICE = 'cuda' if torch.cuda.is_available() else 'mps'
print("Device: ", DEVICE)

Device:  cuda


In [19]:
config = {
    'batch_size': 256, # Increase this if your GPU can handle it
    'lr': 0.005,
    'epochs': 50, # 20 epochs is recommended ONLY for the early submission - you will have to train for much longer typically.
    'num_classes': 8631, #Dataset contains 8631 classes for classification, reduce this number if you want to train on a subset, but only for train dataset and not on val dataset
    'augument': True, 
    "patience": 40, 
    "save_every": 2, 
    "weight_decay": 1e-5,    # 1e-5 for run4. 
    "logits": "raw", 
    "note": "[(3, 64, 128), (3, 128, 256), (2, 1024, 2048)]"
}

def clean_cache(device):
    if device == "mps":
        torch.mps.empty_cache()
    elif device == "cuda":
        torch.cuda.empty_cache()

In [3]:
# Prepare Data for Classification

num_workers = 10

train_transforms = create_transforms(augment=config['augument'])
val_transforms   = create_transforms(augment=False)

cls_train_dataset = ImageDataset(root="../autodl-tmp/data/HW2p2_S25/cls_data/train", transform=train_transforms)
cls_val_dataset   = ImageDataset(root="../autodl-tmp/data/HW2p2_S25/cls_data/dev", transform=val_transforms)
cls_test_dataset  = ImageDataset(root="../autodl-tmp/data/HW2p2_S25/cls_data/test", transform=val_transforms)

assert cls_train_dataset.classes == cls_val_dataset.classes == cls_test_dataset.classes, "Class mismatch!"

# Dataloaders
cls_train_loader = DataLoader(cls_train_dataset, batch_size=config['batch_size'], shuffle=True,  num_workers=num_workers, pin_memory=True)
cls_val_loader   = DataLoader(cls_val_dataset,   batch_size=config['batch_size'], shuffle=False, num_workers=num_workers, pin_memory=True)
cls_test_loader  = DataLoader(cls_test_dataset,  batch_size=config['batch_size'], shuffle=False, num_workers=num_workers, pin_memory=True)

/root/miniconda3/envs/py311/lib/python3.11/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
Loading dataset:   0%|          | 0/431550 [00:00<?, ?it/s]

Loading dataset: 100%|██████████| 43155/43155 [00:00<00:00, 765068.20it/s]


In [4]:
# Prepare Data for Verificatin 

num_workers = 8

# Datasets
ver_val_dataset  = ImagePairDataset(root="../autodl-tmp/data/HW2p2_S25/ver_data", pairs_file="../autodl-tmp/data/HW2p2_S25/val_pairs.txt", transform=create_transforms(augment=False))
ver_test_dataset = ImagePairDataset(root="../autodl-tmp/data/HW2p2_S25/ver_data", pairs_file="../autodl-tmp/data/HW2p2_S25/test_pairs.txt", transform=create_transforms(augment=False), test=True)

# Dataloader
ver_val_loader   = DataLoader(ver_val_dataset,  batch_size=config['batch_size'], shuffle=False, num_workers=num_workers, pin_memory=True) # 8
ver_test_loader  = DataLoader(ver_test_dataset, batch_size=config['batch_size'], shuffle=False, num_workers=num_workers, pin_memory=True) # 8

Loading image pairs: 100%|██████████| 6000/6000 [00:02<00:00, 2439.61it/s]


In [5]:
# Double-check dataset/dataloaders work as expected
print("Number of classes    : ", len(cls_train_dataset.classes))
print("No. of train images  : ", cls_train_dataset.__len__())
print("Shape of image       : ", cls_train_dataset[0][0].shape)
print("Batch size           : ", config['batch_size'])
print("Train batches        : ", cls_train_loader.__len__())
print("Val batches          : ", cls_val_loader.__len__())

print("batches = {}".format(len(cls_train_loader)))

Number of classes    :  8631
No. of train images  :  431550
Shape of image       :  torch.Size([3, 112, 112])
Batch size           :  256
Train batches        :  1686
Val batches          :  169
batches = 1686


In [ ]:
show_cls_dataset_samples(cls_train_loader, cls_val_loader, cls_test_loader)

In [ ]:
show_ver_dataset_samples(ver_val_loader)

In [20]:
# Load Model

# 47: [(3, 64, 128), (4, 128, 256), (6, 256, 512), (2, 1024, 2048)]
# 49: [(3, 64, 64), (4, 64, 96), (6, 96, 192), (3, 192, 384)]

ResNeXt47 = ResNeXt(params=[(3, 64, 128), (3, 128, 256), (2, 1024, 2048)],
                            cardinality=32, 
                            num_classes=8631, DEVICE=DEVICE, 
                            cosine_similarity=(config["logits"] == "cosine")).to(DEVICE)
model = ResNeXt47

# model = Network(config["num_classes"])
model_name = "ResNeXt"

In [21]:
clean_cache(DEVICE)
inputs, _ = next(iter(cls_train_loader))
model.apply_init(inputs.to(DEVICE), initialize_weights)
# model.apply(initialize_weights)

print(inputs.shape)

criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.1)
# criterion = Arcface()

optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"], weight_decay=config["weight_decay"])
# optimizer = torch.optim.AdamW(model.parameters(), lr=config["lr"], weight_decay=config["weight_decay"])


scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=config["lr"], 
    total_steps = config["epochs"] * len(cls_train_loader), 
    pct_start = 0.1, # 0.15
    anneal_strategy="cos", 
    final_div_factor=1     # 0.0001
)

scaler = torch.amp.GradScaler('cuda', enabled=True) if DEVICE == "cuda" else None
clean_cache(DEVICE)
gc.collect()

start=0
best_val_acc = 0

torch.Size([256, 3, 112, 112])


In [8]:
# wandb login 
wandb.login(key="232fdceeae8cfc797a73cea836fb0c4a6199a7ae")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: peng_qiu (11785-DL) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
# For test
wandb.unwatch(model)
test_Trainer = Trainer(config["epochs"], criterion, optimizer, scheduler,
                       config["patience"], config["save_every"], model_name, device=DEVICE, scaler=scaler)
test_Trainer.fit(model, cls_train_loader, cls_val_loader, ver_val_loader, log_epoch=False, log_batch=False, save_best=False,
                 checkpoints=False)

In [22]:
name = f"{model_name}_ablation_3"

In [23]:
# Create wandb run
run = wandb.init(
    name    = name, ### set run names
    reinit  = True, ### Allows reinitalizing runs when re-running this cell
    #id     = "", ### Insert specific run id here if resuming a previous run
    #resume = "must", ### need this to resume previous runs, but comment out reinit = True when using this
    project = "HW2P2", ### Project name
    group=f"{model_name}", 
    config=config
)

batch,▇▇▂█▃▇▃█▂▃▃▂▄▃█▃▇▂▂▅▆▆▂▅▇▅▄▆▇▅▅▆▂▂█▇▇▁▁█
batch_acc,▁▁▁▅▅▆▆▆▆▇▇▇▇▇▇▇████████████████████████
batch_loss,██▆▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_cls_acc,▁▁▂▃▄▆▆▆▇▇▇▇▇▇██████████████████████████
train_loss,█▇▅▄▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_cls_acc,▁▁▃▄▅▆▇▇▇▇▇▇▇▇▇█████████████████████████
valid_loss,█▆▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
valid_ret_acc,▁▆▇▇█▇▇█████████████████████████████████
batch,1680
batch_acc,98.82812


In [ ]:
clean_cache(DEVICE)
gc.collect()

trainer = Trainer(config["epochs"], criterion, optimizer, scheduler, 
                       config["patience"], config["save_every"], name, 
                       start=start, best_val_acc=best_val_acc, 
                       device=DEVICE, scaler=scaler)
trainer.fit(model, cls_train_loader, cls_val_loader, ver_val_loader, save_best=True, checkpoints=True, log_freq=20, early_stopping=False)


Epoch 1/50


Train:   0%|          | 0/1686 [00:00<?, ?it/s]

/root/kaggle_face/scripts/train.py:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():  # This implements mixed precision. Thats it!


	Train Cls. Acc 0.0174%	Train Cls. Loss 9.0473	 Learning Rate 0.0002000
	Val Cls. Acc 0.0647%	Val Cls. Loss 8.7463


{'ACC': np.float64(66.8), 'EER': 34.62321792260694, 'AUC': np.float64(72.20539454783349), 'TPRs': [('TPR@FPR=1e-4', np.float64(0.40733197556008144)), ('TPR@FPR=5e-4', np.float64(0.40733197556008144)), ('TPR@FPR=1e-3', np.float64(0.40733197556008144)), ('TPR@FPR=5e-3', np.float64(0.6109979633401221)), ('TPR@FPR=5e-2', np.float64(13.64562118126273))]}
Val Ret. Acc 66.8000%

Epoch 2/50


...

	Train Cls. Acc 99.3319%	Train Cls. Loss 1.5224	 Learning Rate 0.0002000
	Val Cls. Acc 86.0570%	Val Cls. Loss 2.2066


{'ACC': np.float64(96.3), 'EER': 4.276985743379523, 'AUC': np.float64(98.83322196391632), 'TPRs': [('TPR@FPR=1e-4', np.float64(87.57637474541752)), ('TPR@FPR=5e-4', np.float64(87.57637474541752)), ('TPR@FPR=1e-3', np.float64(87.57637474541752)), ('TPR@FPR=5e-3', np.float64(90.020366598778)), ('TPR@FPR=5e-2', np.float64(96.13034623217924))]}
Val Ret. Acc 96.3000%

Epoch 38/50


Train:   0%|          | 0/1686 [00:00<?, ?it/s]/root/kaggle_face/scripts/train.py:31: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():  # This implements mixed precision. Thats it!


	Train Cls. Acc 99.3595%	Train Cls. Loss 1.5178	 Learning Rate 0.0002000
	Val Cls. Acc 86.1402%	Val Cls. Loss 2.2138


{'ACC': np.float64(96.7), 'EER': 3.8696537678178635, 'AUC': np.float64(98.84482572353443), 'TPRs': [('TPR@FPR=1e-4', np.float64(87.37270875763747)), ('TPR@FPR=5e-4', np.float64(87.37270875763747)), ('TPR@FPR=1e-3', np.float64(87.37270875763747)), ('TPR@FPR=5e-3', np.float64(92.4643584521385)), ('TPR@FPR=5e-2', np.float64(96.33401221995926))]}
Val Ret. Acc 96.7000%


In [11]:
# Predict
scores = test_epoch_ver(model, ver_test_loader)

with open("verification_submission_4.csv", "w+") as f:
    f.write("ID,Label\n")
    for i in range(len(scores)):
        f.write("{},{}\n".format(i, scores[i]))

In [ ]:
"----------------------------------------------------------------------"
"----------------------------Resume Run--------------------------------"
"----------------------------------------------------------------------"

In [10]:
# Load checkpoint
# checkpoint = torch.load(f"../autodl-fs/checkpoints/{name}/model_epoch_18.pth", weights_only=False)

checkpoint = torch.load("../autodl-fs/checkpoints/ResNeXt47_5_run_2/model_epoch_18.pth", weights_only=False)


model.load_state_dict(checkpoint["model_state_dict"])
# optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
# scheduler.load_state_dict(checkpoint["scheduler_state_dict"])

start=0
best_val_acc = 0.1

In [ ]:
wandb.init(
    name    = name,
    reinit  = True,
    id      = 'y5hgwxz2',   ### ID for the run
    resume  = "must", 
    project = "HW2P2",
    group=f"{model_name}",
    config=config
)

In [ ]:
with torch.no_grad():
    outputs = model(inputs)
    print(torch.isnan(outputs['feats']).any())

for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"{name}: mean = {param.data.mean().item():.4f}, std = {param.data.std().item():.4f}")

In [ ]:
valid_epoch_ver(model, ver_val_loader, DEVICE)

In [ ]:
batch = next(iter(ver_test_loader))
print("Batch type:", type(batch))
print("Element type:", type(batch[0]))
if isinstance(batch[0], dict):
    print("Element keys:", batch[0].keys())
else:
    print("Batch element is not a dict.")


In [ ]:
sample = next(iter(ver_val_loader))

# Print the number of items returned
print("Number of returned items:", len(sample))
print(len(ver_val_loader))